## Using VLLM embedings for retrieval tests

In [1]:
import sys
base_dir = "/home/risa/retrieval/beir/"
sys.path.append(base_dir)

from transformers import AutoTokenizer
from beir.retrieval.models import VLLMEmbed

In [2]:
embedder = VLLMEmbed(
    model_path="IEITYuan/Yuan-embedding-2.0-en",
    prompts={
        "query": "Instruct: Given a web search query, retrieve relevant passages that answer the query\nQuery:",
        "document": "",
    },
    pooling="last"
)
tokenizer = AutoTokenizer.from_pretrained("IEITYuan/Yuan-embedding-2.0-en")

INFO 02-08 23:09:26 [utils.py:261] non-default args: {'convert': 'embed', 'dtype': 'bfloat16', 'disable_log_stats': True, 'enforce_eager': True, 'max_lora_rank': None, 'pooler_config': PoolerConfig(pooling_type='LAST', seq_pooling_type='LAST', tok_pooling_type=None, normalize=None, dimensions=None, enable_chunked_processing=None, max_embed_len=None, softmax=None, activation=None, use_activation=False, logit_bias=None, step_tag_id=None, returned_token_ids=None), 'model': 'IEITYuan/Yuan-embedding-2.0-en'}
INFO 02-08 23:09:27 [config.py:772] Found sentence-transformers modules configuration.
INFO 02-08 23:09:27 [config.py:799] Found pooling configuration.
INFO 02-08 23:09:27 [model.py:812] Resolved `--runner auto` to `--runner pooling`. Pass the value explicitly to silence this message.
INFO 02-08 23:09:27 [model.py:541] Resolved architecture: Qwen3ForCausalLM
INFO 02-08 23:09:27 [model.py:1561] Using max model len 32768
INFO 02-08 23:09:27 [scheduler.py:226] Chunked prefill is enabled wi

[W208 23:09:49.367650993 socket.cpp:209] [c10d] The hostname of the client socket cannot be retrieved. err=-3


(EngineCore_DP0 pid=111624) INFO 02-08 23:09:49 [parallel_state.py:1423] rank 0 in world size 1 is assigned as DP rank 0, PP rank 0, PCP rank 0, TP rank 0, EP rank N/A
(EngineCore_DP0 pid=111624) WARNING 02-08 23:09:50 [interface.py:470] Using 'pin_memory=False' as WSL is detected. This may slow down the performance.
(EngineCore_DP0 pid=111624) INFO 02-08 23:09:50 [gpu_model_runner.py:4033] Starting to load model IEITYuan/Yuan-embedding-2.0-en...
(EngineCore_DP0 pid=111624) INFO 02-08 23:09:51 [cuda.py:364] Using FLASH_ATTN attention backend out of potential backends: ('FLASH_ATTN', 'FLASHINFER', 'TRITON_ATTN', 'FLEX_ATTENTION')
(EngineCore_DP0 pid=111624) INFO 02-08 23:09:51 [weight_utils.py:567] No model.safetensors.index.json found in remote.


Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  9.37it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  9.35it/s]
(EngineCore_DP0 pid=111624) 


(EngineCore_DP0 pid=111624) INFO 02-08 23:09:53 [default_loader.py:291] Loading weights took 1.29 seconds
(EngineCore_DP0 pid=111624) INFO 02-08 23:09:53 [gpu_model_runner.py:4130] Model loading took 1.12 GiB memory and 2.569748 seconds
(EngineCore_DP0 pid=111624) INFO 02-08 23:09:54 [gpu_worker.py:356] Available KV cache memory: 20.23 GiB
(EngineCore_DP0 pid=111624) INFO 02-08 23:09:54 [kv_cache_utils.py:1307] GPU KV cache size: 189,424 tokens
(EngineCore_DP0 pid=111624) INFO 02-08 23:09:54 [kv_cache_utils.py:1312] Maximum concurrency for 32,768 tokens per request: 5.78x
(EngineCore_DP0 pid=111624) INFO 02-08 23:09:59 [core.py:272] init engine (profile, create kv cache, warmup model) took 6.18 seconds
(EngineCore_DP0 pid=111624) INFO 02-08 23:10:01 [vllm.py:624] Asynchronous scheduling is enabled.
(EngineCore_DP0 pid=111624) WARNING 02-08 23:10:01 [vllm.py:669] Inductor compilation was disabled by user settings, optimizations settings that are only active during inductor compilation w

In [21]:
embeddings = embedder.encode_corpus(["This is a test", "NOOOOOOOO", "Not good", "Good", "Bad," "Hello", "How are you", "I don't feel soo good."])

  0%|          | 0/1 [00:00<?, ?it/s]

[2026-02-08 23:18:57] INFO vllm.py:150: Encoding 7 texts...


Adding requests:   0%|          | 0/7 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/7 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

In [23]:
norms = embeddings.norm(p=2, dim=1).reshape(-1, 1)
inner_product = embeddings @ embeddings.T
norm_product = norms @ norms.T
cosine_similarity = inner_product / norm_product
print(cosine_similarity)

tensor([[1.0010, 0.7168, 0.6836, 0.6650, 0.7510, 0.7642, 0.6587],
        [0.7168, 1.0000, 0.7974, 0.7339, 0.7642, 0.7437, 0.7373],
        [0.6836, 0.7974, 1.0000, 0.8257, 0.7871, 0.7412, 0.7925],
        [0.6650, 0.7339, 0.8257, 0.9995, 0.7568, 0.7539, 0.6909],
        [0.7510, 0.7642, 0.7871, 0.7568, 1.0000, 0.7642, 0.7070],
        [0.7642, 0.7437, 0.7412, 0.7539, 0.7642, 1.0000, 0.7124],
        [0.6587, 0.7373, 0.7925, 0.6909, 0.7070, 0.7124, 0.9995]],
       device='cuda:0', dtype=torch.float16)
